In [ ]:
!pip install -qU langchain-huggingface langchain-core transformers accelerate pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 490.2/490.2 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 82.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 79.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.3 which is incompatible.


In [ ]:
import pandas as pd
import re
import torch
import gc
from tqdm import tqdm
from google.colab import files
import os

from transformers import pipeline
from langchain_huggingface import HuggingFacePipeline
from langchain_core.prompts import PromptTemplate
os.environ["HF_TOKEN"] = ""

In [ ]:
def load_data_from_url(url):
    print(f"Загрузка данных из {url}...")
    try:
        df = pd.read_csv(url)
        print(f"Успешно загружено {len(df)} строк")
        return df
    except Exception as e:
        print(f"Ошибка загрузки: {e}")
        return None

DEV_DATA_URL = "https://raw.githubusercontent.com/1kirill81/ML_lab1/refs/heads/main/LR1_dev_mini.csv"
ANSWER_DATA_URL = "https://raw.githubusercontent.com/1kirill81/ML_lab1/refs/heads/main/LR1_dev_mini_answers.csv"


df_questions = load_data_from_url(DEV_DATA_URL)
df_answers = load_data_from_url(ANSWER_DATA_URL)


def parse_options(options_str):
    if pd.isna(options_str) or not isinstance(options_str, str):
        return []

    text = str(options_str).strip()


    if text.startswith('[') and text.endswith(']'):
        text = text[1:-1].strip()


    if not text:
        return []


    pattern = r'[\'"]([^\'"]*?)[\'"]'
    matches = re.findall(pattern, text)

    if matches:
        return matches


    parts = re.split(r'\s{2,}', text)
    return [p.strip(' \'"') for p in parts if p.strip()]


# Задаем списку ответов читаемы вид
def format_options_for_display(options_list, max_length=100):
    if not options_list:
        return "Нет вариантов"

    formatted = []
    for i, opt in enumerate(options_list):
        # Обрезаем длинные варианты
        if len(opt) > max_length:
            opt = opt[:max_length-3] + "..."
        formatted.append(f"{i}. {opt}")

    return "\n".join(formatted)


# Функция для вытягивания номера ответа
def extract_answer_from_text(text):
    if not isinstance(text, str):
        return -1

    text = text.strip()

    # Ищем цифры в тексте
    numbers = re.findall(r'\b\d+\b', text)

    if not numbers:
        return -1

    # Берем последнее число (обычно это финальный ответ)
    try:
        answer = int(numbers[-1])
        if 0 <= answer <= 99:  # Проверяем разумный диапазон
            return answer
    except:
        pass

    return -1


# Функция загрузки модельки и создания промта
def init_model_and_promt(model_name="microsoft/phi-2"):
  print(f"\n\nЗагрузка модели: {model_name}\n\n")

  model = model_name

  hf_pipeline = pipeline(
    "text-generation",
    model = model,
    device_map='auto',
    max_new_tokens=512,
    do_sample=True,
    temperature=0.1
  )

  llm = HuggingFacePipeline(pipeline=hf_pipeline)

  prompt_template = PromptTemplate(
            input_variables=["category", "question", "options"],
            template="""Ответь на вопрос, но перед этим обдумай кажды вариант ответа из предложенных
Категория: {category}
Вопрос: {question}
Варианты: {options}
Правильный вариант: """
          )

  return llm, prompt_template


def procces_question(llm, prompt_template, category, question, options_str, question_num=0):
  try:
    options_list = parse_options(options_str)
    foptions = format_options_for_display(options_list)
    prompt = prompt_template.format(
        category=category,
        question=question,
        options=foptions
    )

    response = llm.invoke(prompt)
    promt_template2 = '''Тебе будет предоставлен ход решения задачи. Необходимо извлечь финальный ответ и написать только его числом.
Решение:{decision}

Ответ:'''

    promt2 = promt_template2.format(
        decision=response
    )

    response2 = llm.invoke(promt2)

    answer_number = extract_answer_from_text(response2)

    return answer_number


  except Exception as e:
        print(f"Ошибка в вопросе {question_num}: {e}")
        return -1


def main():


  llm, prompt_template = init_model_and_promt()
  row = df_questions.iloc[3]
  answer = procces_question(
                llm=llm,
                prompt_template=prompt_template,
                category=row.get("category"),
                question=row.get("question"),
                options_str=str(row.get("options")),
                question_num=0
            )
  print(f"Вопрос {3}: {answer}")

  # all_ans = []

  # print("Запуск обратоки. Кол-во вопросов: ", len(df_questions))

  # for i in range(len(df_questions)):
  #   print(f"Обработка вопроса {i}")
  #   row = df_questions.iloc[i]
  #   answer = procces_question(
  #               llm=llm,
  #               prompt_template=prompt_template,
  #               category=row.get("category"),
  #               question=row.get("question"),
  #               options_str=str(row.get("options")),
  #               question_num=0
  #           )
  #   all_ans.append(answer[1])
  #   print("="*30)
  #   print(f"Вопрос {i}: {answer[1]}\nПолный ответ: \n{answer[0]}")
  #   print("="*30)

  # print(all_ans)


main()


Загрузка данных из https://raw.githubusercontent.com/1kirill81/ML_lab1/refs/heads/main/LR1_dev_mini.csv...
Ошибка загрузки: name 'pd' is not defined
Загрузка данных из https://raw.githubusercontent.com/1kirill81/ML_lab1/refs/heads/main/LR1_dev_mini_answers.csv...
Ошибка загрузки: name 'pd' is not defined


Загрузка модели: microsoft/phi-2




NameError: name 'pipeline' is not defined